# 02 – Data Cleaning

**Goal:** Clean and prepare the dataset for funnel analysis

**Steps**
1. Remove missing CustomerIDs (~25%)
2. Separate returns (~2%)
3. Create calculated columns
4. Remove outliers and bad data
5. Save clean datasets


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


In [2]:
raw_path = Path("../data/raw/Online Retail.xlsx")

df = pd.read_excel(raw_path, engine="openpyxl")
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")

print(f"✅ Dataset loaded: {len(df):,} rows × {df.shape[1]} columns")

✅ Dataset loaded: 541,909 rows × 8 columns


In [3]:
missing_customers = df["CustomerID"].isna().sum()
missing_pct = missing_customers / len(df) * 100

print(f"Missing CustomerIDs: {missing_customers:,} ({missing_pct:.2f}%)")

df = df[df["CustomerID"].notna()].copy()
print(f"Remaining rows: {len(df):,}")

Missing CustomerIDs: 135,080 (24.93%)
Remaining rows: 406,829


In [4]:
df["IsReturn"] = df["Quantity"] < 0

returns = df[df["IsReturn"]].copy()
purchases = df[df["Quantity"] > 0].copy()

print(f"Purchases: {len(purchases):,}")
print(f"Returns: {len(returns):,}")

Purchases: 397,924
Returns: 8,905


In [5]:
purchases["TotalPrice"] = purchases["Quantity"] * purchases["UnitPrice"]

purchases["Year"] = purchases["InvoiceDate"].dt.year
purchases["Month"] = purchases["InvoiceDate"].dt.month
purchases["YearMonth"] = purchases["InvoiceDate"].dt.to_period("M").astype(str)
purchases["Hour"] = purchases["InvoiceDate"].dt.hour

purchases["CustomerSegment"] = np.where(
    purchases["Country"] == "United Kingdom",
    "UK",
    "International"
)

print("Calculated columns added")

Calculated columns added


In [6]:
before = len(purchases)

purchases = purchases[purchases["UnitPrice"] > 0]
purchases = purchases[purchases["Description"].notna()]
purchases = purchases[purchases["TotalPrice"] > 0]

qty_cap = purchases["Quantity"].quantile(0.99)
price_cap = purchases["TotalPrice"].quantile(0.99)

purchases = purchases[purchases["Quantity"] <= qty_cap]
purchases = purchases[purchases["TotalPrice"] <= price_cap]

print(f"Removed {before - len(purchases):,} rows")
print(f"Clean rows remaining: {len(purchases):,}")

Removed 5,805 rows
Clean rows remaining: 392,119


In [8]:
processed = Path("../data/processed")
processed.mkdir(parents=True, exist_ok=True)

purchases.to_csv(processed / "cleaned_purchases.csv", index=False)
returns.to_csv(processed / "returns.csv", index=False)

print("✅ Files saved")

✅ Files saved


In [9]:
print(f"Purchases saved: {len(purchases):,} rows")
print(f"Returns saved: {len(returns):,} rows")

Purchases saved: 392,119 rows
Returns saved: 8,905 rows
